In [4]:
import numpy as np
import pandas as pd

import talib
from datetime import datetime

from binance.client import Client
from apscheduler.schedulers.blocking import BlockingScheduler
import os, time

In [5]:
## Vì để thực hiện giao dịch trực tiếp với API sàn của em, thì nó sẽ cần tiền mặt thật để đổ vào do đó em đã chuyển qua dạng hàm test đặc trưng.

In [10]:
from aux import *
client = Client('JLCON0ZJ9v4zX595o2W1rlvSIhIercTu2ugNLCMPZ88QAt8zvSe2vY3LDiiRjKMB','JMnozKSUlep8mBrXCW1b7mAdHDl5qcYv3GDfNOquyifCdbD4QmGXILR9A5V0FgKf')
symbols = ['BTC']
start_n_hours_ago = 320
balance_unit = 'USDT'
first = True
balance = 100
unittoken = 0
BUY_AMOUNT_USDT = 100
symbol = 'BTC'

while True:
    if (datetime.now().second % 10 == 0) or first:
        if (datetime.now().minute == 0 and datetime.now().second == 10) or first:
            # refresh data
            df = gather_data(320)
            states = get_states(df)
            print('Current state of the market:')
            print(states)
            print('\n')


            if states == 'uptrend' and qty == 0 :
                qty = balance/df['close']
            if states == 'dowtrend' and qty > 0:
                balance = qty*df['close']
                qty = 0





NameError: name 'start_str' is not defined